In [1]:
from glob import glob
import os
import numpy as np
import open3d as o3d
import meshplot as mp

In [148]:
noise_list = ['03001627_1bb81d54471d7c1df51f77a6d7299806_model_27_2020-09-24-14-53-43_sketch',
             '03001627_2a0f4fe08cec3d25680876614ed35a7f_model_20_2020-09-16-12-58-52_sketch']

# 1. Load raw sketches

Load original sketches from `raw_dir`, please replace it with your save directory of the original sketch obj files.

In [2]:
raw_dir = '/vol/vssp/SF_datasets/multiview/3VS/datasets/FineGrained_3DSketch/3DV_SketchyVR/sketch_obj/original_obj'
objs = glob(os.path.join(raw_dir, '*.obj'))

In [3]:
len(objs)

1559

Load timestamp file as a N x 4 array. Each line is [x, y, z, t] where t is the related time 

In [103]:
file_name = noise_list[1]
obj_file = os.path.join(raw_dir, file_name + '.obj')
timestamp = obj_file.replace('_sketch.obj', '_timestamp.txt')

In [105]:
time_array = np.loadtxt(timestamp, delimiter=' ')

Each line is formatted as `[x, y, z, t]`

In [8]:
time_array

array([[ 2.705314e-01, -1.067828e-03,  2.715820e-01,  0.000000e+00],
       [ 2.705989e-01, -1.491884e-03,  2.720228e-01,  2.319336e-02],
       [ 2.706159e-01, -1.681919e-03,  2.722431e-01,  3.674316e-02],
       ...,
       [ 2.199945e-01,  1.301780e-01,  3.130999e-01,  5.344360e+00],
       [ 2.206306e-01,  1.306313e-01,  3.143868e-01,  5.356201e+00],
       [ 2.207554e-01,  1.308748e-01,  3.141899e-01,  5.365601e+00]])

In [106]:
xyz = time_array[:, :3]

In [20]:
xyz.shape

(2456, 3)

Visualize sketch. There is a noisy stroke in the corner.

In [107]:
d = mp.subplot(xyz, c=xyz[:, 1], s=[1, 2, 0], shading={"point_size": 0.03})

## Normalization

Normalize original sketch to unit bounding box:

In [81]:
def normalize_to_box(input):
    """
    normalize point cloud to unit bounding box
    center = (max - min)/2
    scale = max(abs(x))
    input: pc [N, P, dim] or [P, dim]
    output: pc, centroid, furthest_distance

    From https://github.com/yifita/pytorch_points
    """
    if len(input.shape) == 2:
        axis = 0
        P = input.shape[0]
        D = input.shape[1]
    elif len(input.shape) == 3:
        axis = 1
        P = input.shape[1]
        D = input.shape[2]
    else:
        raise ValueError()
    
    if isinstance(input, np.ndarray):
        maxP = np.amax(input, axis=axis, keepdims=True)
        minP = np.amin(input, axis=axis, keepdims=True)
        centroid = (maxP+minP)/2
        input = input - centroid
        furthest_distance = np.amax(np.abs(input), axis=(axis, -1), keepdims=True)
        input = input / furthest_distance
    elif isinstance(input, torch.Tensor):
        maxP = torch.max(input, dim=axis, keepdim=True)[0]
        minP = torch.min(input, dim=axis, keepdim=True)[0]
        centroid = (maxP+minP)/2
        input = input - centroid
        in_shape = list(input.shape[:axis])+[P*D]
        furthest_distance = torch.max(torch.abs(input).reshape(in_shape), dim=axis, keepdim=True)[0]
        furthest_distance = furthest_distance.unsqueeze(-1)
        input = input / furthest_distance
    else:
        raise ValueError()

    return input, centroid, furthest_distance

In [108]:
norm, centroid, furthest_distance = normalize_to_box(xyz)

In [83]:
norm

array([[-0.93684761, -0.03273783, -0.3521659 ],
       [-0.93656578, -0.0315883 , -0.35299554],
       [-0.93687171, -0.0314144 , -0.35282918],
       ...,
       [-0.91375618, -0.15544851,  0.33422232],
       [-0.91070476, -0.16100946,  0.33635392],
       [-0.90717295, -0.16893056,  0.34096978]])

In [92]:
norm.shape

(5367, 3)

# Filtering

`radius_outlier_removal` removes points that have few neighbors in a given sphere around them. Two parameters can be used to tune the filter to your data: 

- `nb_points`, which lets you pick the minimum amount of points that the sphere should contain.
- `radius`, which defines the radius of the sphere that will be used for counting the neighbors.

For more details please refer to data:http://www.open3d.org/docs/0.12.0/tutorial/geometry/pointcloud_outlier_removal.html

Load sketch as point cloud and remove outlier:

In [109]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
nb_points = int(norm.shape[0]*0.1)
cl, ind = pcd.remove_radius_outlier(nb_points=nb_points, radius=0.2) # ind is the index list of all remained points
new_pc = xyz[ind] # points list after filtering

In [110]:
new_pc

array([[0.02788896, 0.2493026 , 0.1143481 ],
       [0.02769374, 0.2489777 , 0.1147905 ],
       [0.02752536, 0.2487068 , 0.114978  ],
       ...,
       [0.2114903 , 0.1242627 , 0.2672291 ],
       [0.2120175 , 0.1238131 , 0.2675781 ],
       [0.2124599 , 0.1232184 , 0.267924  ]])

In [111]:
new_pc.shape

(4002, 3)

Visualize filtered sketch, the noisy stroke is removed after filtering.

In [112]:
d = mp.subplot(new_pc, c=new_pc[:, 1], s=[1, 2, 0], shading={"point_size": 0.03})

Split strokes according to timestamps

In [114]:
t = time_array[:, 3]

In [130]:
t.shape

(4044,)

In [126]:
begin = np.where(t==0)[0] # the start index of each stroke

In [127]:
begin

array([   0,   42,  265,  459,  704,  916, 1101, 1216, 1328, 1509, 1666,
       1790, 1841, 1891, 1957, 2033, 2086, 2238, 2320, 2392, 2453, 2538,
       2622, 2736, 2838, 2943, 3029, 3140, 3290, 3433, 3561, 3690, 3828,
       3925])

In [143]:
filtered_strokes = []
start = 0
for i, start in enumerate(begin):
    end = begin[i+1] if i < len(begin) - 1 else t.shape[0] - 1
    middle = int((start + end)/2)
    if middle in ind:
#         strokes.append([start, end])
        filtered_strokes.append(time_array[start:end])

points list of the last stroke of the filtered ssketch:

In [140]:
filtered_strokes[-1]

array([[0.04634205, 0.1089906 , 0.2697861 ],
       [0.04699868, 0.1089036 , 0.2701499 ],
       [0.04742217, 0.108754  , 0.2701908 ],
       [0.04797826, 0.1084742 , 0.2701976 ],
       [0.04864131, 0.1080806 , 0.270251  ],
       [0.04889285, 0.1078953 , 0.2702953 ],
       [0.04929682, 0.1077179 , 0.2703888 ],
       [0.05008909, 0.107486  , 0.2705843 ],
       [0.05136194, 0.107269  , 0.2708175 ],
       [0.05252812, 0.1072152 , 0.2709261 ],
       [0.05411363, 0.1072442 , 0.27087   ],
       [0.05572303, 0.1073117 , 0.2707042 ],
       [0.0571284 , 0.1073857 , 0.2705097 ],
       [0.05856383, 0.1074383 , 0.2703438 ],
       [0.06003307, 0.1075073 , 0.2702675 ],
       [0.06160605, 0.1076045 , 0.2703311 ],
       [0.06319246, 0.1077736 , 0.2704539 ],
       [0.06429505, 0.1079244 , 0.2706179 ],
       [0.06629045, 0.108162  , 0.2708459 ],
       [0.06838974, 0.1084369 , 0.271058  ],
       [0.07031802, 0.108755  , 0.2711976 ],
       [0.07234646, 0.1090832 , 0.2713326 ],
       [0.

# Save obj file

In [135]:
def get_edge_list(vertice_array):
    edge_list = []
    p_count = 0
    for point_list in vertice_array:
        p_count += 1
        for i in range(len(point_list) - 1):
            edge_list.append([p_count, p_count + 1])
            p_count += 1
    return edge_list

In [134]:
def save_obj_file(file_path, v, l, is_utf8=False):
    if is_utf8:
        f = codecs.open(file_path, 'w', 'utf-8')
    else:
        f = open(file_path, 'w')
    for item in v:
        f.write('v %.6f %.6f %.6f \n' % (item[0], item[1], item[2]))
    for item in l:
        f.write('l %s %s \n' % (item[0], item[1]))
    if len(l) >= 1:
        f.write('l %s %s \n' % (l[-1][0], l[-1][1]))
    f.close()

In [142]:
save_path = 'filtered.obj'
vertice_list = [item for stroke in filtered_strokes[:, :3] for item in stroke]
edge_list = get_edge_list(filtered_strokes[:, :3])

save_obj_file(save_path, vertice_list, edge_list)

Now you can visualize the obj file `filtered.obj` using any 3D software you like!

# Save timestamp file

We can use the time info of the filtered sketch to replace the original one:

In [146]:
save_path = 'filtered_timestamp.txt'
timestamp_list = [item for stroke in filtered_strokes for item in stroke]
np.savetxt(save_path, timestamp_list, fmt='%0.8f')